# customer segmentation op vlak van pageviews (WIP)

In [41]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, select, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
load_dotenv()
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)


In [71]:
#get Inschrijvingen, Contact en account
print(metadata.tables.keys())
Pageviews = metadata.tables['Pageviews']
Contact = metadata.tables['Contactfiche']
Account = metadata.tables['Account']
query = select(Pageviews.c.PageTitle, Contact.c.ContactpersoonId).join(Contact, Pageviews.c.ContactId == Contact.c.ContactpersoonId).join(Account, Contact.c.AccountId == Account.c.AccountId)
result = conn.execute(query)

#convert to dataframe
df = pd.DataFrame(result.fetchall())

df["rating"] = 1


dict_keys(['Account', 'AccountActiviteitscode', 'Activiteitscode', 'AccountFinancieleData', 'AfspraakAccount', 'AfspraakContact', 'Contactfiche', 'Persoon', 'AfspraakVereistContact', 'Campagne', 'ContactficheFunctie', 'Functie', 'Gebruiker', 'InfoEnKlachten', 'Inschrijving', 'Mailing', 'Pageviews', 'Visits', 'SendEmailClicks', 'Sessie', 'SessieInschrijving', 'sysdiagrams', 'WebContent'])


In [72]:
df.head()

,PageTitle,ContactpersoonId,rating
0,Weer hogere loonindexering op komst in 2024 | ...,71C7CDFA-379C-EB11-811E-001DD8B72B62,1
1,Nieuwjaarsinvitito 2023 | Voka,2A04879B-1141-EC11-8125-001DD8B72B61,1
2,Netwerkevents | Voka,1E9BCBC0-9025-EC11-8124-001DD8B72B61,1
3,Marketing | Voka,3F1EB182-8E3C-E811-80EF-001DD8B72B62,1
4,Bedrijfsafval: dit verandert op 1 januari | Voka,5073196D-6611-E811-80EF-001DD8B72B61,1


In [73]:
df_pivot = pd.pivot_table(df, index='ContactpersoonId', columns='PageTitle', values='rating', fill_value = 0)
df_pivot

PageTitle,""" Hitting the ground running"" | Voka","""2.330 zonnepanelen besparen 432 ton CO2"" | Voka","""2023 vertoont eerste tekenen van lichte vooruitgang"" | Voka","""5.000 dagen werken voor pensioen? We mogen de lat écht hoger leggen"" | Voka","""6.000 mensen extra aan de slag in regio in half jaar tijd"" | Voka","""Aansluiten bij een lerend netwerk van Voka is de investering meer dan waard!"" | Voka","""Al 100 hectare industriegebied aan Albertkanaal geschrapt"" | Voka","""Al meer dan 35.000 woningen voorzien van een duurzame energieoplossing"" | Voka","""Al meer dan 35000 woningen voorzien van een duurzame energieoplossing"" | Voka","""Alles op alles, zowel privé als op het werk"" | Voka",...,vzw - interest Bryo StandUp | Voka,vzw Stroom - Francis Mertens | Voka,vzw_Contact_ENG | Voka,webinar_plato.key_.pdf (5 MB),website,website van het Nationaal Forum,welt_Tekengebied 1.png,wisselkoersrisico's,working paper,| Voka
ContactpersoonId,,,,,,,,,,,,,,,,,,,,,
00013C8A-6F1C-E211-9DAA-005056B06EB4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00017E48-212C-EC11-8127-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002EDD7-2405-E511-ABE8-005056B06EB4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006CEC0-931A-EB11-8117-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000C5DD4-016A-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FF9A9C90-DC1A-E711-80E4-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FFAA375C-B44A-E711-80E7-001DD8B72B61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FFD3EBB0-F16C-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
Contact = '000C5DD4-016A-E111-B43A-00505680000A'

#get all pages visited by account that contain the string Zoekresultaten
df_pivot.filter(regex='Zoekresultaten')

PageTitle,"Zoekresultaten voor "" 1 pagina"" | Voka","Zoekresultaten voor "" Family Business Happening"" | Voka","Zoekresultaten voor "" Ref.0327518"" | Voka","Zoekresultaten voor "" Uitreiking Voka Charter Duurzaam Ondernemen Voka"" | Voka","Zoekresultaten voor "" Vermogens- & successieplanning"" | Voka","Zoekresultaten voor "" arry callebaut "" | Voka","Zoekresultaten voor "" digilab cybersecurity"" | Voka","Zoekresultaten voor "" erwelkoming nieuwe leden"" | Voka","Zoekresultaten voor "" financieel management voor niet financielen"" | Voka","Zoekresultaten voor "" het optimaliseren van je bedrijfs"" | Voka",...,"Zoekresultaten voor ""zonnepanelen"" | Voka","Zoekresultaten voor ""zorgeloos leven"" | Voka","Zoekresultaten voor ""zuid oost limburg"" | Voka","Zoekresultaten voor ""zuid-afrika"" | Voka","Zoekresultaten voor ""zuid-oost limburg"" | Voka","Zoekresultaten voor ""zweden"" | Voka","Zoekresultaten voor ""zwinstreek"" | Voka","Zoekresultaten voor ""zwitserland"" | Voka","Zoekresultaten voor ""´plato"" | Voka",Zoekresultaten | Voka
ContactpersoonId,,,,,,,,,,,,,,,,,,,,,
00013C8A-6F1C-E211-9DAA-005056B06EB4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00017E48-212C-EC11-8127-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002EDD7-2405-E511-ABE8-005056B06EB4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006CEC0-931A-EB11-8117-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000C5DD4-016A-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FF9A9C90-DC1A-E711-80E4-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FFAA375C-B44A-E711-80E7-001DD8B72B61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FFD3EBB0-F16C-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
#remove the columns containing the string Zoekresultaten
df_pivot = df_pivot.loc[:,~df_pivot.columns.str.contains('Zoekresultaten')]

In [76]:
account = '99C841DF-C768-E111-B43A-00505680000A'

#get all pages visited by account that contain the string Zoekresultaten
df_pivot.filter(regex='Zoekresultaten')

PageTitle
ContactpersoonId
00013C8A-6F1C-E211-9DAA-005056B06EB4
00017E48-212C-EC11-8127-001DD8B72B62
0002EDD7-2405-E511-ABE8-005056B06EB4
0006CEC0-931A-EB11-8117-001DD8B72B62
000C5DD4-016A-E111-B43A-00505680000A
...
FF9A9C90-DC1A-E711-80E4-001DD8B72B62
FFAA375C-B44A-E711-80E7-001DD8B72B61
FFD3EBB0-F16C-E111-B43A-00505680000A


In [77]:
similarity_matrix = cosine_similarity(df_pivot)
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=df_pivot.index, columns=df_pivot.index)
similarity_matrix_df

ContactpersoonId,00013C8A-6F1C-E211-9DAA-005056B06EB4,00017E48-212C-EC11-8127-001DD8B72B62,0002EDD7-2405-E511-ABE8-005056B06EB4,0006CEC0-931A-EB11-8117-001DD8B72B62,000C5DD4-016A-E111-B43A-00505680000A,000CDD37-8DA1-EB11-811A-001DD8B72B61,000E8357-036C-E111-B43A-00505680000A,0010617B-4806-E211-88C4-005056B06EC4,00107805-3B76-E511-895A-005056B06EC4,0011C821-5BD4-ED11-A7C7-6045BD895BFB,...,FF4712DE-D46B-E111-B43A-00505680000A,FF4A49E4-9360-E711-80E7-001DD8B72B62,FF4F0250-C8A9-EC11-983F-0022488489AB,FF5B3D87-0021-EC11-8124-001DD8B72B61,FF76B927-E283-E611-80E3-001DD8B72B62,FF9A9C90-DC1A-E711-80E4-001DD8B72B62,FFAA375C-B44A-E711-80E7-001DD8B72B61,FFD3EBB0-F16C-E111-B43A-00505680000A,FFD66075-2663-EC11-8F8F-000D3A2E7886,FFD685FA-1F7B-ED11-81AD-6045BD895CDC
ContactpersoonId,,,,,,,,,,,,,,,,,,,,,
00013C8A-6F1C-E211-9DAA-005056B06EB4,1.00000,0.0,0.0,0.0,0.21693,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
00017E48-212C-EC11-8127-001DD8B72B62,0.00000,1.0,0.0,0.0,0.00000,0.000000,0.288675,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
0002EDD7-2405-E511-ABE8-005056B06EB4,0.00000,0.0,1.0,0.0,0.00000,0.000000,0.408248,0.0,0.0,0.333333,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
0006CEC0-931A-EB11-8117-001DD8B72B62,0.00000,0.0,0.0,1.0,0.00000,0.166667,0.000000,0.0,0.0,0.192450,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
000C5DD4-016A-E111-B43A-00505680000A,0.21693,0.0,0.0,0.0,1.00000,0.140028,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.09167,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FF9A9C90-DC1A-E711-80E4-001DD8B72B62,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,0.0
FFAA375C-B44A-E711-80E7-001DD8B72B61,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,1.0,0.0,0.0,0.0
FFD3EBB0-F16C-E111-B43A-00505680000A,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,0.0


In [78]:
account = "0006CEC0-931A-EB11-8117-001DD8B72B62"

#get amount of 1 values from df_pivot for this account
pages_visited = df_pivot.loc[account].value_counts()[1]
print(pages_visited)
#num of silimar users
n = 10
#set a threshold for similarity
user_similarity_threshold =  0.3
# Get top n similar users
similar_users = similarity_matrix_df[similarity_matrix_df[account]>user_similarity_threshold][account].sort_values(ascending=False).drop(account)[:n]
similar_users

12


ContactpersoonId
651348F3-190F-43DA-98C2-B021A78CCB8F    0.516398
2FC8138C-ED0E-E211-9DAA-005056B06EB4    0.516398
16B9337F-24EE-EC11-BB3D-0022488584A4    0.510310
21E2D40E-BDCA-EC11-A7B5-00224881D030    0.510310
2F2A15B7-12B8-ED11-83FF-6045BD895BFB    0.500000
300BEAD6-B0A7-ED11-AAD1-6045BD8956C9    0.500000
58D9FD78-E8D4-EB11-8124-001DD8B72B62    0.500000
6930FB84-CBB9-ED11-83FF-6045BD895D85    0.492366
5D9740EE-FF6B-E111-B43A-00505680000A    0.471405
0D6CD93C-056B-E111-B43A-00505680000A    0.471405
Name: 0006CEC0-931A-EB11-8117-001DD8B72B62, dtype: float64

In [79]:
#get accounts with more than 10 pages visited
accounts = df_pivot[df_pivot.sum(axis=1)>10]
accounts

PageTitle,""" Hitting the ground running"" | Voka","""2.330 zonnepanelen besparen 432 ton CO2"" | Voka","""2023 vertoont eerste tekenen van lichte vooruitgang"" | Voka","""5.000 dagen werken voor pensioen? We mogen de lat écht hoger leggen"" | Voka","""6.000 mensen extra aan de slag in regio in half jaar tijd"" | Voka","""Aansluiten bij een lerend netwerk van Voka is de investering meer dan waard!"" | Voka","""Al 100 hectare industriegebied aan Albertkanaal geschrapt"" | Voka","""Al meer dan 35.000 woningen voorzien van een duurzame energieoplossing"" | Voka","""Al meer dan 35000 woningen voorzien van een duurzame energieoplossing"" | Voka","""Alles op alles, zowel privé als op het werk"" | Voka",...,vzw - interest Bryo StandUp | Voka,vzw Stroom - Francis Mertens | Voka,vzw_Contact_ENG | Voka,webinar_plato.key_.pdf (5 MB),website,website van het Nationaal Forum,welt_Tekengebied 1.png,wisselkoersrisico's,working paper,| Voka
ContactpersoonId,,,,,,,,,,,,,,,,,,,,,
0006CEC0-931A-EB11-8117-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000C5DD4-016A-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000CDD37-8DA1-EB11-811A-001DD8B72B61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001BDC5F-CB6B-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002D23A6-866A-E111-B43A-00505680000A,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F8EF1E6D-F8E0-E911-8106-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F9DF0004-5768-EA11-810F-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAABA27F-BB41-EA11-810B-001DD8B72B61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
#for each account get the top n similar users
similar_users = {}
for account in accounts.index:
    similar_users[account] = similarity_matrix_df[similarity_matrix_df[account]>user_similarity_threshold][account].sort_values(ascending=False).drop(account)[:n]
similar_users

{'0006CEC0-931A-EB11-8117-001DD8B72B62': ContactpersoonId
 651348F3-190F-43DA-98C2-B021A78CCB8F    0.516398
 2FC8138C-ED0E-E211-9DAA-005056B06EB4    0.516398
 16B9337F-24EE-EC11-BB3D-0022488584A4    0.510310
 21E2D40E-BDCA-EC11-A7B5-00224881D030    0.510310
 2F2A15B7-12B8-ED11-83FF-6045BD895BFB    0.500000
 300BEAD6-B0A7-ED11-AAD1-6045BD8956C9    0.500000
 58D9FD78-E8D4-EB11-8124-001DD8B72B62    0.500000
 6930FB84-CBB9-ED11-83FF-6045BD895D85    0.492366
 5D9740EE-FF6B-E111-B43A-00505680000A    0.471405
 0D6CD93C-056B-E111-B43A-00505680000A    0.471405
 Name: 0006CEC0-931A-EB11-8117-001DD8B72B62, dtype: float64,
 '000C5DD4-016A-E111-B43A-00505680000A': ContactpersoonId
 042CC29B-27B5-E911-8104-001DD8B72B62    0.420084
 396A5CE5-A885-EC11-8D21-6045BD8F772B    0.396059
 6F28AB25-6973-E111-B43A-00505680000A    0.363803
 6CB2EFE7-386A-E111-B43A-00505680000A    0.363803
 05B73A70-726B-E111-B43A-00505680000A    0.342997
 4BA1A0B0-6DB5-E611-80E3-001DD8B72B62    0.342997
 D37591C1-8BF6-EB11-812